# WDN Pipeline (Colab)
Simulación → Entrenamiento → Inferencia

Este notebook asume que subiste `wdn_project_final.zip` y lo vas a descomprimir aquí.

In [ ]:
# 🧰 Setup: descomprimir paquete y crear carpetas básicas
!unzip -o wdn_project_final.zip -d .
!mkdir -p data/raw data/processed results
!ls -la

## (Opcional) Subir CTOWN.INP
Si ya lo tienes en tu Drive, puedes omitir esta celda y colocar el archivo manualmente en `data/raw/CTOWN.INP`.

In [ ]:
from google.colab import files
up = files.upload()  # selecciona CTOWN.INP si lo vas a usar
if 'CTOWN.INP' in up:
    !mv CTOWN.INP data/raw/
!ls -l data/raw

## 1) Simular y generar dataset

In [ ]:
!python simulate.py --config ctown_example.yaml
!ls -l data/processed

## 2) Entrenar la IA (MLP)

In [ ]:
!python train.py \
  --npz data/processed/training_set_ctown_demo.npz \
  --outdir artifacts_p_demo \
  --mode p \
  --x_names "JUNCTION-101,JUNCTION-205"


## 3) Métricas de evaluación

In [ ]:
import json, pandas as pd, numpy as np
from pathlib import Path
m_path = Path('artifacts_p_demo/metrics.json')
if m_path.exists():
    m = json.loads(m_path.read_text())
    mae, rmse, best = m.get('test_mae'), m.get('test_rmse'), m.get('best_val_mse')
    escala_ref = 1.0  # cambia por una escala natural o baseline
    eff = 100.0*max(0.0, 1.0 - (rmse/escala_ref)) if rmse is not None else None
    df = pd.DataFrame({
        'Métrica': ['MAE (test)', 'RMSE (test)', 'Best Val MSE', 'Eficiencia (%)'],
        'Valor': [mae, rmse, best, eff]
    })
    df
else:
    print('No se encontró artifacts_p_demo/metrics.json. Corre la celda de entrenamiento primero.')

## 4) Inferencia y vista previa

In [ ]:
!python infer.py \
  --model artifacts_p_demo/best_model.pt \
  --scalers artifacts_p_demo/scalers.npz \
  --feature_names artifacts_p_demo/x_feature_names.json \
  --mode p \
  --csv data/processed/timeseries_ctown_demo.csv \
  --dout 2 \
  --map \
  --save results/p_pred.csv

import pandas as pd
pred = pd.read_csv('results/p_pred.csv')
ref  = pd.read_csv('data/processed/labels_min_cost_ctown_demo.csv')
pd.concat([ref.add_prefix('ref_').head(10), pred.add_prefix('pred_').head(10)], axis=1)

### ¡Listo!
Con esto tienes todo el flujo: simulación → entrenamiento → inferencia + vista de resultados.
Ajusta el YAML y los nombres de nodos según tu red real.